In [1]:
import numpy as np
import pandas as pd
from quadprog_wrapper import solve_quadprog

In [2]:
def kernel_svm_train(data, labels):
    g_mat = (np.dot(data.T, data) + 1) ** 2
    n = g_mat.shape[0]
    hes = np.outer(labels, labels) * g_mat
    w = np.ones(n)
    coef = np.zeros((1, n))
    coef[0, :] = labels

    # Call quadratic program with provided inputs.
    a = solve_quadprog(hes, w, coef, np.zeros(1), None, None, np.zeros(n), 1.0)

    # process optimized alphas to only store support vectors and alphas that have nonnegligible support
    error = 1e-6
    vec_idx = a > error
    v = data[:, vec_idx]
    w_fix = a[vec_idx]
    l = labels[vec_idx]

    # find all alphas that represent points on the decision margin
    marg_a = np.logical_and(a > error, a < 1.0 - error)

    # compute the bias value
    if np.any(marg_a):
        b = np.mean(labels[marg_a].T - (a * labels).T.dot(g_mat[:, marg_a]))
    else:
        # there were no support vectors on the margin (this should only happen due to numerical errors)
        b = 0
        
    return v, w_fix, l, b

In [3]:
def svm_predict(data, w, b, v, l):
    gram = (np.dot(data.T, v) + 1) ** 2
    scores = gram.dot(w * l) + b
    return scores.ravel()

In [4]:
def relabel(num, sample_set):
    binary = sample_set.copy(deep=True)
    binary.loc[binary['label'] != num, 'label'] = -1
    binary.loc[binary['label'] == num, 'label'] = 1
    return binary

In [5]:
def training(data):
    raw_set = data.copy(deep=True)
    raw_set_label = raw_set['label']
    raw_set_label = raw_set_label.values
    raw_set_data = raw_set.drop(columns=['label'], inplace=True)
    raw_set_data = raw_set.values/255.0
    [vec, w, vec_lb, b] = kernel_svm_train(raw_set_data.T, raw_set_label)
    return w, b, vec, vec_lb

In [6]:
trainData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')

In [7]:
group = trainData.groupby('label')
zero = pd.DataFrame(group.get_group(0))
one = pd.DataFrame(group.get_group(1))
two = pd.DataFrame(group.get_group(2))
three = pd.DataFrame(group.get_group(3))
four = pd.DataFrame(group.get_group(4))
five = pd.DataFrame(group.get_group(5))
six = pd.DataFrame(group.get_group(6))
seven = pd.DataFrame(group.get_group(7))
eight = pd.DataFrame(group.get_group(8))
nine = pd.DataFrame(group.get_group(9))

In [8]:
zero_data = zero.drop(columns=['label'])
one_data = one.drop(columns=['label'])
two_data = two.drop(columns=['label'])
three_data = three.drop(columns=['label'])
four_data = four.drop(columns=['label'])
five_data = five.drop(columns=['label'])
six_data = six.drop(columns=['label'])
seven_data = seven.drop(columns=['label'])
eight_data = eight.drop(columns=['label'])
nine_data = nine.drop(columns=['label'])

In [9]:
count = 200
rdm_zero = zero.sample(count)
rdm_one = one.sample(count)
rdm_two = two.sample(count)
rdm_three = three.sample(count)
rdm_four = four.sample(count)
rdm_five = five.sample(count)
rdm_six = six.sample(count)
rdm_seven = seven.sample(count)
rdm_eight = eight.sample(count)
rdm_nine = nine.sample(count)

In [10]:
sample_set = rdm_zero
sample_set = sample_set.append(rdm_one)
sample_set = sample_set.append(rdm_two)
sample_set = sample_set.append(rdm_three)
sample_set = sample_set.append(rdm_four)
sample_set = sample_set.append(rdm_five)
sample_set = sample_set.append(rdm_six)
sample_set = sample_set.append(rdm_seven)
sample_set = sample_set.append(rdm_eight)
sample_set = sample_set.append(rdm_nine)

In [11]:
class_zero = relabel(0, sample_set)
class_one = relabel(1, sample_set)
class_two = relabel(2, sample_set)
class_three = relabel(3, sample_set)
class_four = relabel(4, sample_set)
class_five = relabel(5, sample_set)
class_six = relabel(6, sample_set)
class_seven = relabel(7, sample_set)
class_eight = relabel(8, sample_set)
class_nine = relabel(9, sample_set)

In [12]:
w = [None] * 10
b = [None] * 10
v = [None] * 10
l = [None] * 10
[w[0], b[0], v[0], l[0]] = training(class_zero)
[w[1], b[1], v[1], l[1]] = training(class_one)
[w[2], b[2], v[2], l[2]] = training(class_two)
[w[3], b[3], v[3], l[3]] = training(class_three)
[w[4], b[4], v[4], l[4]] = training(class_four)
[w[5], b[5], v[5], l[5]] = training(class_five)
[w[6], b[6], v[6], l[6]] = training(class_six)
[w[7], b[7], v[7], l[7]] = training(class_seven)
[w[8], b[8], v[8], l[8]] = training(class_eight)
[w[9], b[9], v[9], l[9]] = training(class_nine)

In [13]:
testCut = testData.copy(deep=True)
label = testCut['label']
testCut.drop(columns = ['label'], inplace=True)
testCut = testCut.values/255.0

correct = 0
for j in range(testCut.shape[0]):
    piece = testCut[j]
    result = [None] * 10
    for i in range(10):
        result[i] = svm_predict(piece, w[i], b[i], v[i], l[i])
    value = result.index(max(result))
    if (value == label[j]):
        correct = correct + 1

accuracy = correct / label.shape[0]
accuracy

0.9415